In [14]:
import json
import pandas as pd
from astropy import table 
from astropy.table import Table, Column, MaskedColumn, vstack, unique, join
from astropy.coordinates import SkyCoord
from astropy import units as u
from IPython.display import clear_output
from astropy.io import ascii
import numpy as np
from astroquery.vizier import Vizier
import collections
from collections import Counter
import sqlutilpy
import utilfuncs as uf
import dataloader as dl
import addsurveys as adds
import sys
sys.path.append('/home/jupyter-tingli/pythoncode/')
import crossmatcher
import warnings
warnings.filterwarnings('ignore')

# Booleans to add the respective additional surveys
Add_SAGA = True
Add_JINA = True

# Alex Ji's catalogue provides upper and lower bound measurements
# If the abundance value is an upper bound, the error is 9999
# If the  abundance value is a lower bound, the error is -9999
Add_AlexJi = True

# Adds corresponding Gaia IDs for stars. If there is no ID, value is -9999
Add_Gaia = True

# Path of file where cleaned paper files will be stored
Temp_path = 'testing'

# File name for where manually compiled catalogue will be stored
Lit_name = 'file_name_1'

# File name for the full catalogue
Litplus_name = 'file_name_2'

#Condition: Paper_list cannot be empty
paper_list = {
    'Duggan 2018': {
    'tables': [1],
    'tag': 'J/ApJ/869/50',
    'path': Temp_path + '/Duggan 2018.json',
    'name': 'Duggan 2018',
    'star_header': 'Name',
    'gal_header': 'Target',
    'paper_type': 'Vizier'
    },
    'Kirby 2010':{
    'tables': [0],
    'tag': 'J/ApJS/191/352',
    'path': Temp_path + '/Kirby 2010.json',
    'name': 'Kirby 2010',
    'star_header': 'Name',
    'gal_header': 'dSph',
    'paper_type': 'Vizier'
    },
    'Kirby 2009': {
    'tables': [0],
    'tag': 'J/ApJ/705/328',
    'path': Temp_path + '/Kirby 2009.json',
    'name': 'Kirby 2009',
    'star_header': 'recno',
    'gal_header': None,
    'gal_name': 'Scl',
    'paper_type': 'Vizier'
    },
    'Kirby 2015': {
    'tables': [2],
    'tag': 'J/ApJ/801/125',
    'path': Temp_path + '/Kirby 2015.json',
    'name': 'Kirby 2015',
    'star_header': 'Name',
    'gal_header': 'Syst',
    'paper_type': 'Vizier'
    },
    'Skuladottir 2019': {
    'tables': [0],
    'tag': 'J/A+A/631/A171',
    'path': Temp_path + '/Skuladottir 2019.json',
    'name': 'Skuladottir 2019',
    'star_header': 'Star',
    'gal_header': None,
    'gal_name': 'Scl',
    'paper_type': 'Vizier'
    },
    'Theler 2020': {
    'tables': [0, 6, 7, 8],
    'tag': 'J/A+A/642/A176',
    'path': Temp_path + '/Theler.json',
    'name': 'Theler 2020',
    'star_header': 'ID',
    'gal_header': None,
    'gal_name': 'Sex',
    'paper_type': 'Vizier'
    },
    'de los Reyes 2020': {
    'tables': [0],
    'tag': 'J/ApJ/891/85',
    'path': Temp_path + '/de los Reyes 2020.json',
    'name': 'de los Reyes 2020',
    'star_header': 'ID',
    'gal_header': 'AName',
    'paper_type': 'Vizier'
    },
    'Kirby 2017-A':{
    'tables': [1, 2],
    'tag': 'J/ApJ/834/9',
    'path': Temp_path + '/Kirby 2017-A.json',
    'name': 'Kirby 2017-A',
    'star_header': 'Name',
    'gal_header': 'Gal',
    'paper_type': 'Vizier'
    },
    'Kirby 2017':{
    'tables': [0, 1],
    'tag': 'J/ApJ/838/83',
    'path': Temp_path + '/Kirby 2017.json',
    'name': 'Kirby 2017',
    'star_header': ['__KCS2015_', '__MIC2016_'],
    'gal_header': None,
    'gal_name': 'TriII',
    'paper_type': 'Vizier'
    },
    'Aoki 2013': {
    'tables': [0],
    'tag': 'J/AJ/145/13',
    'path': Temp_path + '/Aoki 2013.json',
    'name': 'Aoki 2013',
    'star_header': 'Name',
    'gal_header': None,
    'gal_name': 'Metal Poor Halo',
    'paper_type': 'Vizier'
    },
    'Norris 2013': {
    'tables': [0],
    'tag': 'J/ApJ/762/25',
    'path': Temp_path + '/Norris 2013.json',
    'name': 'Norris 2013',
    'star_header': 'Name',
    'gal_header': None,
    'gal_name': 'Metal Poor Halo',
    'paper_type': 'Vizier'
    },
    'Yong 2013': {
    'tables': [3],
    'tag': 'J/ApJ/762/26',
    'path': Temp_path + '/Yong 2013.json',
    'name': 'Yong 2013',
    'star_header': 'Name',
    'gal_header': None,
    'gal_name': 'Metal Poor Halo',
    'paper_type': 'Vizier'
    },
    'Roederer 2014-B': {
    'tables': [0, 5],
    'tag': 'J/AJ/147/136',
    'path': Temp_path + '/Roederer 2014-B.json',
    'name': 'Roederer 2014-B',
    'star_header': 'Name',
    'gal_header': None,
    'gal_name': 'Metal Poor Halo',
    'paper_type': 'Vizier'
    },
    'Jacobson 2015': {
    'tables': [0, 1],
    'tag': 'J/ApJ/807/171',
    'path': Temp_path + '/Jacobson 2015.json',
    'name': 'Jacobson 2015',
    'star_header': 'ID',
    'gal_header': None,
    'gal_name': 'Metal Poor Halo',
    'paper_type': 'Vizier'
    },
    'Lombardo 2022': {
    'tables': [0, 1],
    'tag': 'J/A+A/665/A10',
    'path': Temp_path + '/Lombardo 2022.json',
    'name': 'Lombardo 2022',
    'star_header': 'Star',
    'gal_header': None,
    'gal_name': 'Metal Poor Halo',
    'paper_type': 'Vizier'
    },
    'Nissen 2024':{
    'tables': [0],
    'tag': 'J/A+A/682/A116',
    'path': Temp_path + '/Nissen 2024.json',
    'name': 'Nissen 2024',
    'star_header': 'Star',
    'gal_header': None,
    'gal_name': 'Metal Poor Halo',
    'paper_type': 'Vizier'
    },
    'Li 2022':{
    'tables': [0],
    'tag': 'J/ApJ/931/147',
    'path': Temp_path + '/Li 2022.json',
    'name': 'Li 2022',
    'star_header': 'ID',
    'gal_header': None,
    'gal_name': 'Metal Poor Halo',
    'paper_type': 'Vizier'
    },
    'Shetrone 2001':{
    'tables': [0],
    'tag': '2001ApJ...548..592S',
    'path': Temp_path + '/Shetrone 2001.json',
    'name': 'Shetrone 2001',
    'star_header': 'MAIN_ID',
    'gal_header': None,
    'gal_name': 'Metal Poor Halo',
    'paper_type': 'Simbad'
    },
    'Kirby 2013':{
    'tables': [1],
    'tag': 'J/ApJ/779/102',
    'path': Temp_path + '/Kirby 2013.json',
    'name': 'Kirby 2013',
    'star_header': 'Name',
    'gal_header': 'Gal',
    'paper_type': 'Vizier'
    },
    'Cohen 2013':{
    'tables': [0],
    'tag': 'J/ApJ/778/56',
    'path': Temp_path + '/Cohen 2013.json',
    'name': 'Cohen 2013',
    'star_header': 'ID',
    'gal_header': None,
    'gal_name': 'Metal Poor Halo',
    'paper_type': 'Vizier'
    },
    'Reichert 2020':{
    'tables': [0],
    'tag': 'J/A+A/641/A127',
    'path': Temp_path + '/Reichert 2020.json',
    'name': 'Reichert 2020',
    'star_header': 'ID',
    'gal_header': 'Galaxy',
    'paper_type': 'Vizier'
    },
    'Yoon 2016':{
    'tables': [0,1],
    'tag': 'J/ApJ/833/20',
    'path': Temp_path + '/Yoon 2016.json',
    'name': 'Yoon 2016',
    'star_header': 'Name',
    'gal_header': None,
    'gal_name': 'Metal Poor Halo',
    'paper_type': 'Vizier'
    }
}

#Custom stack assumes at least a column of 'Name', 'RAJ2000', and 'DEJ2000'. Can include any other columns which are
# assumed to have the naming convention applied
# It is also assumed that RA and DEC are both in degrees
custom = {
    'extra_1':{
        'Name': 'Test',
        'Table_name': 'variable'
    }
}

In [15]:
# Load in all papers
curr_count = 0
tag_coll = []
for paper in paper_list:
    clear_output(wait=True)
    tag = paper_list[paper]['tag']
    tables = paper_list[paper]['tables']
    path = paper_list[paper]['path']
    name = paper_list[paper]['name']
    star_header = paper_list[paper]['star_header']
    gal_header = paper_list[paper]['gal_header']
    gal_name = paper_list[paper].get('gal_name', None)
    paper_type = paper_list[paper].get('paper_type')
    tag_coll.append([name, tag])

    print(f'Retrieving {name}...')
    tmp = adds.retrieve_catalog_bulk(tag, tables, name, star_header, paper_type, gal_header, gal_name, path)
    
# Stack all papers together
init = json.load(open(Temp_path+'/Duggan 2018.json', "r"))
lit_cat = Table.from_pandas(uf.make_dataframe(init, False))
lit_cat['Star_ID'] = Column([f"{i:05.0f}" for i in range(1, len(lit_cat) + 1)])
entry_count = len(lit_cat)
first_go = True


#Assume that paper is used in addition to camps so that star ids are already decided

for paper in paper_list:
    if first_go:
        first_go = False
        continue
    lit_cat, paper_list, paper, entry_count = adds.stack(lit_cat, paper_list, paper, entry_count)

adds.lit_bibs(lit_cat, tag_coll)
adds.galaxies(lit_cat)
adds.fix_types(lit_cat)
adds.fix_names(lit_cat)
lit_cat['Solar Abundance'] = ['nan'] * len(lit_cat)
lit_cat['Resolution'] = ['nan'] * len(lit_cat)
lit_cat['Facility'] = ['nan'] * len(lit_cat)
#ascii.write(lit_cat, Lit_name, format='csv', fast_writer=False, overwrite=True) 

#if custom != {}:
 #   for extra in custom:
  #      lit_cat,curr_count = adds.cust_stack(extra['Table_name'], lit_cat, extra['Name'], curr_count)
if Add_SAGA:
    lit_cat, curr_count = adds.saga(lit_cat, curr_count)
if Add_AlexJi:
    lit_cat, curr_count = adds.ji(lit_cat, curr_count)
if Add_JINA:
    lit_cat, curr_count = adds.jina(lit_cat, curr_count)
if Add_Gaia:
    adds.gaia(lit_cat)

# this is temporary until the next camps metadata release (when that happens remove the one/two thing)
one = lit_cat[lit_cat['From']!='Yoon 2016']
two = lit_cat[lit_cat['From']=='Yoon 2016']
data = ascii.read('CAMPS_Metadata_Dict.csv')
one.remove_columns(['Paper','Facility','Resolution','Solar Abundance'])
data.remove_columns(['From','Original Order'])
one=join(one, data, keys='Bib', join_type='inner')
one.remove_column('Bib')
one.rename_column('BibT', 'Bib')
two['Resolution'] = two['Resolution'].astype(str)
lit_cat = vstack([one,two])
lit_cat

lit_cat = adds.clean(lit_cat)
lit_cat = lit_cat[np.argsort(lit_cat['Star_ID'])]
lit_cat

Gaia_ID,Star_ID,Name,Paper,Bib,From,RAJ2000,DEJ2000,Object Name,Object Type,log_g,er_log_g,T_eff,er_T_eff,RV,er_RV,V_mic,er_V_mic,Member,S_N,Facility,Resolution,Solar Abundance,Fe_H,er_Fe_H,FeII_H,er_FeII_H,FeI_H,er_FeI_H,M_H,A_C,A_Na,A_Mg,A_Ca,A_Ti,A_Fe,A_FeI,A_FeII,A_Sr,A_Ba,Li_Fe,er_Li_Fe,Be_Fe,er_Be_Fe,BeI_FeI,BeII_FeII,BI_FeI,C_Fe,er_C_Fe,CI_FeI,N_Fe,er_N_Fe,O_Fe,er_O_Fe,OI_FeI,F_Fe,er_F_Fe,Na_Fe,er_Na_Fe,NaI_FeI,Mg_Fe,er_Mg_Fe,MgI_FeI,MgII_FeII,Al_Fe,er_Al_Fe,AlI_FeI,Si_Fe,er_Si_Fe,SiI_FeI,SiII_FeII,P_Fe,er_P_Fe,PI_FeI,S_Fe,er_S_Fe,SI_FeI,K_Fe,er_K_Fe,KI_FeI,Ca_Fe,er_Ca_Fe,CaI_FeI,er_CaI_FeI,CaII_FeII,er_CaII_FeII,Sc_Fe,er_Sc_Fe,ScI_FeI,ScII_FeII,er_ScII_FeII,Ti_Fe,er_Ti_Fe,TiI_FeI,TiII_FeII,er_TiII_FeII,V_Fe,er_V_Fe,VI_FeI,VII_FeII,er_VII_FeII,Cr_Fe,er_Cr_Fe,CrI_FeI,er_CrI_FeI,CrII_FeII,er_CrII_FeII,Mn_Fe,er_Mn_Fe,MnI_FeI,MnII_FeII,er_MnII_FeII,Co_Fe,er_Co_Fe,CoI_FeI,CoII_FeII,Ni_Fe,er_Ni_Fe,NiI_FeI,NiII_FeII,Cu_Fe,er_Cu_Fe,CuI_FeI,CuII_FeII,Zn_Fe,er_Zn_Fe,ZnI_FeI,ZnII_FeII,Ga_Fe,er_Ga_Fe,GaI_FeI,Ge_Fe,er_Ge_Fe,GeI_FeI,As_Fe,er_As_Fe,AsI_FeI,Se_Fe,er_Se_Fe,SeI_FeI,Rb_Fe,er_Rb_Fe,RbI_FeI,Sr_Fe,er_Sr_Fe,SrI_FeI,SrII_FeII,Y_Fe,er_Y_Fe,YII_FeII,Zr_Fe,er_Zr_Fe,ZrI_FeI,ZrII_FeII,Nb_Fe,er_Nb_Fe,NbI_FeI,NbII_FeII,Mo_Fe,er_Mo_Fe,MoI_FeI,MoII_FeII,TcI_FeI,Ru_Fe,er_Ru_Fe,RuI_FeI,Rh_Fe,er_Rh_Fe,RhI_FeI,Pd_Fe,er_Pd_Fe,PdI_FeI,Ag_Fe,er_Ag_Fe,AgI_FeI,Cd_Fe,er_Cd_Fe,CdI_FeI,CdII_FeII,Sn_Fe,er_Sn_Fe,SnI_FeI,Te_Fe,er_Te_Fe,TeI_FeI,Ba_Fe,er_Ba_Fe,BaII_FeII,La_Fe,er_La_Fe,LaI_FeI,LaII_FeII,Ce_Fe,er_Ce_Fe,CeII_FeII,Pr_Fe,er_Pr_Fe,PrII_FeII,Nd_Fe,er_Nd_Fe,NdII_FeII,Sm_Fe,er_Sm_Fe,SmII_FeII,Eu_Fe,er_Eu_Fe,EuI_FeI,EuII_FeII,Gd_Fe,er_Gd_Fe,GdI_FeI,GdII_FeII,Tb_Fe,er_Tb_Fe,TbII_FeII,Dy_Fe,er_Dy_Fe,DyII_FeII,Ho_Fe,er_Ho_Fe,HoII_FeII,Er_Fe,er_Er_Fe,ErII_FeII,Tm_Fe,er_Tm_Fe,TmII_FeII,Yb_Fe,er_Yb_Fe,YbII_FeII,Lu_Fe,er_Lu_Fe,LuII_FeII,Hf_Fe,er_Hf_Fe,HfII_FeII,W_Fe,er_W_Fe,WI_FeI,WII_FeII,Os_Fe,er_Os_Fe,OsI_FeI,OsII_FeII,Ir_Fe,er_Ir_Fe,IrI_FeI,Pt_Fe,er_Pt_Fe,PtI_FeI,Au_Fe,er_Au_Fe,AuI_FeI,Hg_Fe,er_Hg_Fe,HgII_FeII,Pb_Fe,er_Pb_Fe,PbI_FeI,PbII_FeII,Bi_Fe,er_Bi_Fe,BiI_FeI,Th_Fe,er_Th_Fe,ThII_FeII,U_Fe,er_U_Fe,UII_FeII,C2_Fe,CH_Fe,CN_Fe,HF_Fe,NH_Fe,OH_Fe
int64,str13,str30,str23,str61,str18,float64,float64,str15,str9,float64,float64,float64,float64,float64,float64,float64,float64,str32,float64,str177,str32,str289,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,floa

In [17]:
ascii.write(lit_cat, 'CAMPS_Catalogue.csv', format='csv', fast_writer=False, overwrite=True) 

### Run to generate metadata table

In [18]:
lit2 = lit_cat.copy()
lit2['Paper', 'Bib','Facility', 'Resolution', 'Solar Abundance']
lit2 = lit2[np.argsort(lit2['Bib'])]
lit_cut = unique(lit2, keys='Bib', silent = True, keep = 'first')
lit_cut = lit_cut['Paper', 'Bib', 'From', 'Facility', 'Resolution', 'Solar Abundance']
ascii.write(lit_cut, 'CAMPS_Metadata.csv', format='csv', fast_writer=False, overwrite=True) 
lit_cut

Paper,Bib,From,Facility,Resolution,Solar Abundance
str23,str61,str18,str177,str32,str289
Ryan 1991,1991AJ....102..303R,JINA,Coude Echelle Spectrometer (CES) at AAT (3.9m),55000,self
Vanture (1992),1992AJ....103.2035V,Yoon 2016,nan,nan,nan
Kipper & Jorgense,1994A&A...290..148K,Yoon 2016,nan,nan,nan
McWilliam 1995,1995AJ....109.2736M,SAGA,"2D-FRUTTI detector of the du Pont telescope at Las Campanas Observatory (du Pont, 2.5m)",22000,"Moore ( 1966, sst, book, M)"
McWilliam 1995,1995AJ....109.2757M,JINA,"2D-FRUTTI detector of the du Pont telescope at Las Campanas Observatory (du Pont, 2.5m)",22000,"Anders and Grevesse (1989, GeCoA, 53, 197)"
Ryan 1996,1996ApJ...471..254R,JINA,Siding Spring Mountain Observatory Telescope (2.3m),nan,"Anders and Grevesse (1989, GeCoA, 53, 197)"
Norris 1997,1997ApJ...485..370N,JINA,Coude Echelle Spectrometer (CES) at AAT (3.9m),40000,self
Norris 1997,1997ApJ...488..350N,JINA,UCLES on Anglo-Australian 3.9m Telescope,40000,"Anders and Grevesse (1989, GeCoA, 53, 197)"
Norris 1997,1997ApJ...489L.169N,JINA,Coude Echelle Spectrometer (CES) at AAT (3.9m),40000,self


### Run to collapse the table (remove species differentiation)

In [19]:
lit_collapse = lit_cat.copy()
adds.collapse(lit_collapse, Add_SAGA)
ascii.write(lit_collapse, 'CAMPS_Collapsed.csv', format='csv', fast_writer=False, overwrite=True)
lit_collapse

Gaia_ID,Star_ID,Name,Paper,Bib,From,RAJ2000,DEJ2000,Object Name,Object Type,log_g,er_log_g,T_eff,er_T_eff,RV,er_RV,V_mic,er_V_mic,Member,S_N,Facility,Resolution,Solar Abundance,Fe_H,er_Fe_H,M_H,A_C,A_Na,A_Mg,A_Ca,A_Ti,A_Fe,A_FeI,A_FeII,A_Sr,A_Ba,Li_Fe,er_Li_Fe,Be_Fe,er_Be_Fe,B_Fe,C_Fe,er_C_Fe,N_Fe,er_N_Fe,O_Fe,er_O_Fe,F_Fe,er_F_Fe,Na_Fe,er_Na_Fe,Mg_Fe,er_Mg_Fe,Al_Fe,er_Al_Fe,Si_Fe,er_Si_Fe,P_Fe,er_P_Fe,S_Fe,er_S_Fe,K_Fe,er_K_Fe,Ca_Fe,er_Ca_Fe,Sc_Fe,er_Sc_Fe,Ti_Fe,er_Ti_Fe,V_Fe,er_V_Fe,Cr_Fe,er_Cr_Fe,Mn_Fe,er_Mn_Fe,Co_Fe,er_Co_Fe,Ni_Fe,er_Ni_Fe,Cu_Fe,er_Cu_Fe,Zn_Fe,er_Zn_Fe,Ga_Fe,er_Ga_Fe,Ge_Fe,er_Ge_Fe,As_Fe,er_As_Fe,Se_Fe,er_Se_Fe,Rb_Fe,er_Rb_Fe,Sr_Fe,er_Sr_Fe,Y_Fe,er_Y_Fe,Zr_Fe,er_Zr_Fe,Nb_Fe,er_Nb_Fe,Mo_Fe,er_Mo_Fe,Tc_Fe,Ru_Fe,er_Ru_Fe,Rh_Fe,er_Rh_Fe,Pd_Fe,er_Pd_Fe,Ag_Fe,er_Ag_Fe,Cd_Fe,er_Cd_Fe,Sn_Fe,er_Sn_Fe,Te_Fe,er_Te_Fe,Ba_Fe,er_Ba_Fe,La_Fe,er_La_Fe,Ce_Fe,er_Ce_Fe,Pr_Fe,er_Pr_Fe,Nd_Fe,er_Nd_Fe,Sm_Fe,er_Sm_Fe,Eu_Fe,er_Eu_Fe,Gd_Fe,er_Gd_Fe,Tb_Fe,er_Tb_Fe,Dy_Fe,er_Dy_Fe,Ho_Fe,er_Ho_Fe,Er_Fe,er_Er_Fe,Tm_Fe,er_Tm_Fe,Yb_Fe,er_Yb_Fe,Lu_Fe,er_Lu_Fe,Hf_Fe,er_Hf_Fe,W_Fe,er_W_Fe,Os_Fe,er_Os_Fe,Ir_Fe,er_Ir_Fe,Pt_Fe,er_Pt_Fe,Au_Fe,er_Au_Fe,Hg_Fe,er_Hg_Fe,Pb_Fe,er_Pb_Fe,Bi_Fe,er_Bi_Fe,Th_Fe,er_Th_Fe,U_Fe,er_U_Fe,C2_Fe,CH_Fe,CN_Fe,HF_Fe,NH_Fe,OH_Fe
int64,str13,str30,str23,str61,str18,float64,float64,str15,str9,float64,float64,float64,float64,float64,float64,float64,float64,str32,float64,str177,str32,str289,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
-9999,Aqr_00001,1590,Kirby 2013,2013ApJ...779..102K,Manual Compilation,311.7333333,-12.846,Aquarius,dIrr,0.8500000238418579,0.05000000074505806,4452.0,39.0,nan,nan,1.940000057220459,nan,nan,nan,DEep Imaging Multi-Object Spectrograph (DEIMOS) on Keck II,7100,"Asplund et al. (2005, ASPC, 336, 25)",-2.0,0.1599999964237213,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
-9999,Aqr_00001,1590,Kirby 2017,2017ApJ...834....9K,Manual Compilation,311.7333333,-12.846,Aquarius,dIrr,0.9399999976158142,nan,4419.0,nan,nan,nan,nan,nan,Y,nan,DEep Imaging Multi-Object Spectrograph (DEIMOS) on Keck II,7000,"Anders and Grevesse (1989, GeCoA, 53, 197)",-2.0399999618530273,0.1599999964237213,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,